In [1]:
import torch
import itertools
from sklearn import metrics
import math
import time
import csv
import numpy as np
from scipy.optimize import linear_sum_assignment

In [2]:
def accuracy(y_true, y_pred):
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # Find optimal one-to-one mapping between cluster labels and true labels
    row_ind, col_ind = linear_sum_assignment(-contingency_matrix)
    return contingency_matrix[row_ind, col_ind].sum() / np.sum(contingency_matrix)

In [4]:
def GMM_without_Outlier(N,p,m,sigma,w):
    torch.manual_seed(1)
    count = torch.distributions.multinomial.Multinomial(N, w).sample()
    count = count.type(torch.int64)
    y=torch.zeros((N,1))
    x=torch.zeros((N,p))
    for i in range(m):
        pos = torch.randn(1, p)
        obs = pos + std[i]*torch.randn(int(count[i]),p)
        x[sum(count[:i]):sum(count[:i+1]),:]=obs
        y[sum(count[:i]):sum(count[:i+1]),:]=i;
    #neg = torch.randn(int(count[m]), p)
    #x[sum(count[:m]):sum(count[:m+1]),:]=neg
    #y[sum(count[:m]):sum(count[:m+1]),:]=-1
    return x,y

In [5]:
def scrlm(x,n,m,rho,F,device,seed):
    torch.manual_seed(seed)
    thr=0
    [N,p]=x.shape
    nsub = torch.randperm(N)
    nsub = nsub[:n]
    sloss=torch.zeros(n)
    sloss=sloss.to(device)
    loss=torch.cdist(x[nsub,:], x)
    loss=(loss**2)/(p*rho**2)-F
    loss[loss > 0] = 0
    sloss+=torch.sum(loss,1)
    idx = torch.argsort(sloss)
    idx = idx[sloss[idx]<-F]
    counter=0;
    sel= []
    while len(idx) and counter<=m-1:
        i=idx[0]
        sel.append(i)
        a = x[nsub[idx],:]
        b = x[nsub[i],:]
        dist = torch.sqrt(((a-b)**2).sum(axis=1))
        dist = (dist**2)/(p*rho**2)-F
        dist[dist > 0] = 0
        idx = idx[dist>=thr]
        counter=counter+1;
    result = torch.stack(sel)
    centers=x[nsub[result],:]
    return centers

In [8]:
with open('GMM_bound_N_vs_m.csv', 'w', newline='') as csvfile:
    fieldnames = ['p','n','m','a','rho','F','seed','accuracy','time','N','count','N_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    N_list=list(range(20,21))
    for k in range(len(N_list)):
        count = 0
        N = N_list[k]
        m = 2
        p = 3600
        a = 0.7
        w_min = a/m
        w_max = 1.3/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        F=2.5
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        w=torch.linspace(w_min,w_max,m)
        x,y = GMM_without_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'p':p,'n':n,'m':m,'a':a,'rho':rho,'F':F,'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'N': N,'count':count})
        print(N,count)

20 100


In [10]:
with open('GMM_bound_N_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['p','n','m','a','rho','F','seed','accuracy','time','N','count','N_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    N_list=list(range(43,44))
    for k in range(len(N_list)):
        count = 0
        N = N_list[k]
        m = 4
        p = 3600
        a = 0.7
        w_min = a/m
        w_max = 1.3/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        print(n)
        F=2.5
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        #i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        #w = torch.cat((w, i), 0)
        x,y = GMM_without_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'p':p,'n':n,'m':m,'a':a,'rho':rho,'F':F,'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'N': N,'count':count})
        print(N,count)

43
43 100


In [12]:
with open('GMM_bound_N_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['p','n','m','a','rho','F','seed','accuracy','time','N','count','N_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    N_list=list(range(93,94))
    for k in range(len(N_list)):
        count = 0
        N = N_list[k]
        m = 8
        p = 3600
        a = 0.7
        w_min = a/m
        w_max = 1.3/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        print(n)
        F=2.5
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        #i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        #w = torch.cat((w, i), 0)
        x,y = GMM_without_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'p':p,'n':n,'m':m,'a':a,'rho':rho,'F':F,'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'N': N,'count':count})
        print(N,count)

93
93 100


In [15]:
with open('GMM_bound_N_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['p','n','m','a','rho','F','seed','accuracy','time','N','count','N_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    N_list=list(range(201,202))
    for k in range(len(N_list)):
        count = 0
        N = N_list[k]
        m = 16
        p = 3600
        a = 0.7
        w_min = a/m
        w_max = 1.3/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        print(n)
        F=2.5
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        #i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        #w = torch.cat((w, i), 0)
        x,y = GMM_without_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'p':p,'n':n,'m':m,'a':a,'rho':rho,'F':F,'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'N': N,'count':count})
        print(N,count)

201
201 100


In [17]:
with open('GMM_bound_N_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['p','n','m','a','rho','F','seed','accuracy','time','N','count','N_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    N_list=list(range(433,434))
    for k in range(len(N_list)):
        count = 0
        N = N_list[k]
        m = 32
        p = 3600
        a = 0.7
        w_min = a/m
        w_max = 1.3/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        F=2.5
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        #i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        #w = torch.cat((w, i), 0)
        x,y = GMM_without_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'p':p,'n':n,'m':m,'a':a,'rho':rho,'F':F,'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'N': N,'count':count})
        print(N,count)

433 100


In [20]:
with open('GMM_bound_N_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['p','n','m','a','rho','F','seed','accuracy','time','N','count','N_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    N_list=list(range(929,930))
    for k in range(len(N_list)):
        count = 0
        N = N_list[k]
        m = 64
        p = 3600
        a = 0.7
        w_min = a/m
        w_max = 1.3/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        print(n)
        F=2.5
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        #i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        #w = torch.cat((w, i), 0)
        x,y = GMM_without_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'p':p,'n':n,'m':m,'a':a,'rho':rho,'F':F,'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'N': N,'count':count})
        print(N,count)

929
929 100


In [21]:
with open('GMM_bound_N_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['p','n','m','a','rho','F','seed','accuracy','time','N','count','N_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    N_list=list(range(1983,1984))
    for k in range(len(N_list)):
        count = 0
        N = N_list[k]
        m = 128
        p = 3600
        a = 0.7
        w_min = a/m
        w_max = 1.3/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        print(n)
        F=2.5
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        #i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        #w = torch.cat((w, i), 0)
        x,y = GMM_without_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'p':p,'n':n,'m':m,'a':a,'rho':rho,'F':F,'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'N': N,'count':count})
        print(N,count)

1983
1983 100


In [22]:
with open('GMM_bound_N_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['p','n','m','a','rho','F','seed','accuracy','time','N','count','N_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    N_list=list(range(4220,4221))
    for k in range(len(N_list)):
        count = 0
        N = N_list[k]
        m = 256
        p = 3600
        a = 0.7
        w_min = a/m
        w_max = 1.3/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        print(n)
        F=2.5
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        #i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        #w = torch.cat((w, i), 0)
        x,y = GMM_without_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'p':p,'n':n,'m':m,'a':a,'rho':rho,'F':F,'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'N': N,'count':count})
        print(N,count)

4220
4220 100


In [23]:
with open('GMM_bound_N_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['p','n','m','a','rho','F','seed','accuracy','time','N','count','N_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    N_list=list(range(8946,8947))
    for k in range(len(N_list)):
        count = 0
        N = N_list[k]
        m = 512
        p = 3600
        a = 0.7
        w_min = a/m
        w_max = 1.3/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        print(n)
        F=2.5
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        #i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        #w = torch.cat((w, i), 0)
        x,y = GMM_without_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'p':p,'n':n,'m':m,'a':a,'rho':rho,'F':F,'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'N': N,'count':count})
        print(N,count)

8946
8946 100


In [24]:
with open('GMM_bound_N_vs_m.csv', 'a', newline='') as csvfile:
    fieldnames = ['p','n','m','a','rho','F','seed','accuracy','time','N','count','N_','count_']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    N_list=list(range(18905,18906))
    for k in range(len(N_list)):
        count = 0
        N = N_list[k]
        m = 1024
        p = 3600
        a = 0.7
        w_min = a/m
        w_max = 1.3/m
        n = math.ceil(m/0.7*(math.log(m)+math.log(4/0.01)))
        print(n)
        F=2.5
        rho=0.5
        std=torch.linspace(1/16,0.25,m)
        #i = torch.tensor([0.2])
        w=torch.linspace(w_min,w_max,m)
        #w = torch.cat((w, i), 0)
        x,y = GMM_without_Outlier(N,p,m,std,w)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x=x.to(device)
        y=y.to(device)
        run=100
        for j in range(run):
            start_time2 = time.time()
            center=scrlm(x,n,m,rho,F,device,j)
            d=torch.cdist(center,x)
            d = (d**2)/(p*rho**2)-F
            [k,label]=torch.min(d,dim=0)
            label[k>0]=-1
            end_time2 = time.time()
            t1 = end_time2 - start_time2
            acc=accuracy(y.cpu(),label.cpu())
            if acc==1:
                count +=1
            writer.writerow({'p':p,'n':n,'m':m,'a':a,'rho':rho,'F':F,'seed':j,'accuracy':acc,'time':t1})
        writer.writerow({'N': N,'count':count})
        print(N,count)

18905
18905 100
